In [7]:
import os
import subprocess

In [8]:
gpu = 'ga_gpu_output.txt'
cpu = 'ga_cpu_output.txt'
tsp_dir = os.getcwd() + os.sep + 'TSPLIB'

In [9]:
tsp_files_list = os.listdir(tsp_dir)

In [10]:
def execGA(device, filename):
    if(device == 'gpu'):
        return subprocess.run(["./ga_gpu", filename], capture_output=True)
    else:
        return subprocess.run(["./ga_cpu", filename], capture_output=True)

In [11]:
def parseOutputFile(filename):
    fo = open(filename, 'r')
    lines = fo.readlines()
    num_gen = int(lines[0])
    res = {}
    res['g'] = num_gen
    solutions = []
    for i in range(num_gen):
        solutions.append(float(lines[i+1]))
    res['solns'] = solutions
    conv = int(lines[num_gen+1])
    res['converged'] = conv
    time = float(lines[num_gen+2])
    res['time'] = time
    return res